MATH 448  
Homework CH 5  
Andrew Dahlstrom  
4/11/24  


3)       
a)  
This cross validation technique splits the dataset randomly into k folds equally where k is an arbitrarily chosen integer. One of the folds becomes the validation set while the remaining folds are the training set. The model is trained and tested on the validation fold, the performance is measured. This process is repeated until all folds each serve as the validation set once. The performance results are then averaged across the k repetitions.    

b)  
I.  
Advantages relative to the validation set approach include, k-fold uses all parts of the dataset in both training and testing through its process and this process can also reduce the variance compared to the validation set approach which is more dependent on which data was selected for the test or training sets. Disadvantages relative to the validation set approach include increased computational complexity by iterating the process though all k folds. It’s also not as simple to implement, you need to know a good value for k.  

II.   
Advantages relative to LOOCV (Leave one out) is that k-fold has a similar process to LOOCV but is less extreme since LOOCV repeats the process for the number of data observations rather than k times. This means k-fold is much less computationally intensive and also is less sensitive to outliers since an outlier could be the test observation of the LOOCV model. Disadvantages relative to LOOCV include a potentially higher bias because the larger test set (one fold vs one observation) means less training data. Also LOOCV is averaged over many more trials which could lead to a potentially lower variance than k-fold.


In [1]:
# 5
import pandas as pd
import statsmodels.api as sm
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MATH 448/Default.csv')
print(df.head())

  default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879


In [2]:
# a)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# Convert the response to numerical
df['default'] = df['default'].apply(lambda x: 1 if x == 'Yes' else 0)

# Predictor variables
X = df[['income', 'balance']]
# Response variable
y = df['default']
model_train = LogisticRegression(solver='liblinear')

In [3]:
# b)
# Ensure reproducibility
np.random.seed(0)

# Split the dataset into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
from sklearn.metrics import accuracy_score

# Fit logistic regression model on training data
model_train.fit(X_train, y_train)

# Obtain class predictions
predictions_class = model_train.predict(X_val)
print(predictions_class[:100])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [5]:
# Compute the validation set error
val_error = 1 - accuracy_score(y_val, predictions_class)
print(f'Validation set error: {val_error}')

Validation set error: 0.03700000000000003


In [6]:
# c)
for i in range(1, 4):

  # Split the dataset into training and validation
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i)
  # Fit logistic regression model on training data
  model_train.fit(X_train, y_train)

  # Obtain class predictions
  predictions_class = model_train.predict(X_val)

  # Compute the validation set error
  val_error = 1 - accuracy_score(y_val, predictions_class)
  print(f'Validation set error: {val_error}')

Validation set error: 0.02949999999999997
Validation set error: 0.02849999999999997
Validation set error: 0.030000000000000027


The results are all very similar with a correct prediction on validation set of about 97% correct.

In [7]:
# d)
# Create dummy variable for student
df['student'] = df['student'].apply(lambda x: 1 if x == 'Yes' else 0)

# Add student to the predictors
X = df[['income', 'balance', 'student']]
# Response variable
y = df['default']

# Split the dataset into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit logistic regression model on training data
model_train.fit(X_train, y_train)

# Obtain class predictions
predictions_class = model_train.predict(X_val)

# Compute the validation set error
val_error = 1 - accuracy_score(y_val, predictions_class)
print(f'Validation set error: {val_error}')

Validation set error: 0.03700000000000003


Adding the student feature as a predictor did not lead to a reduction in the test error rate.

In [8]:
# 6)
# Set predictors and intercept for sm models
X = sm.add_constant(df[['income', 'balance']])
# Set response
y = df['default']

# a)
# Fit the model using GLM
model = sm.GLM(y, X, family=sm.families.Binomial())
results = model.fit()

# Determine estimated standard errors using summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Fri, 12 Apr 2024   Deviance:                       1579.0
Time:                        05:49:49   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================
"""

In [9]:
# b
# Takes as input data and index of the observations, and outputs
# the coefficient estimates for income and balance
def boot_fn(data, index):
    X = sm.add_constant(data.loc[index, ['income', 'balance']])
    y = data.loc[index, 'default']
    model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
    return model.params[1:]

In [10]:
# c
# Define # of bootstraps
bootstraps = 100
# Coefficients for predictors income and balance
coeffs = np.zeros((bootstraps, 2))

for i in range(bootstraps):
    # Generate random indices
    indices = np.random.choice(df.index, size=len(df), replace=True)
    # Get the bootstrap coefficients
    boot_coeffs = boot_fn(df, indices)
    # Store income and balance coefficients
    coeffs[i,:] = boot_coeffs

# Calculate the standard errors of the coefficients
boot_se = np.std(coeffs, axis=0)
print(f"Bootstrap SE for Income = {boot_se[0]}, for Balance = {boot_se[1]}")

Bootstrap SE for Income = 4.833950265363818e-06, for Balance = 0.0002557994432092796


d)  
In both approaches the estimated standard errors obtained using sm.GLM() and bootstrap are very low. The coefficient error for Income is slightly higher using the GLM and the Balance coefficient SE is higher using bootstrap with 100 interations.

In [11]:
# 7)
weekly = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MATH 448/Weekly.csv')
print(weekly.head())

   Year   Lag1   Lag2   Lag3   Lag4   Lag5    Volume  Today Direction
0  1990  0.816  1.572 -3.936 -0.229 -3.484  0.154976 -0.270      Down
1  1990 -0.270  0.816  1.572 -3.936 -0.229  0.148574 -2.576      Down
2  1990 -2.576 -0.270  0.816  1.572 -3.936  0.159837  3.514        Up
3  1990  3.514 -2.576 -0.270  0.816  1.572  0.161630  0.712        Up
4  1990  0.712  3.514 -2.576 -0.270  0.816  0.153728  1.178        Up


In [13]:
# a
weekly['Direction'] = weekly['Direction'].apply(lambda x: 1 if x == 'Up' else 0)
X = sm.add_constant(weekly[['Lag1', 'Lag2']])
y = weekly['Direction']

# Fit the logistic regression model
model = sm.GLM(y, X, family=sm.families.Binomial())
results = model.fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Fri, 12 Apr 2024   Deviance:                       1488.2
Time:                        05:55:14   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.061      3.599      0.0

In [15]:
# b
# Using all data bit first observation in training set
X_train = X.iloc[1:]
y_train = y.iloc[1:]

model_LOO = sm.GLM(y_train, X_train, family=sm.families.Binomial())
results_LOO = model_LOO.fit()

In [19]:
# c
# Predict the direction of the first observation
pred_prob = results_LOO.predict(X.iloc[0:1])
pred_prob_value = pred_prob.values[0]
pred = 0
if pred_prob_value > 0.5:
  pred = 1
else:
  pred = 0

classification = pred == y.iloc[0]
print(classification)

False


The first observation was not correctly classified.

In [22]:
# d
# Create list to store results
results = []

for i in range(len(weekly)):
    # Leave out the ith observation
    X_train = X.drop(X.index[i])
    y_train = y.drop(y.index[i])
    X_test = X.iloc[i:i+1]

    # Fit the GLM
    model_LOO = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()

    # Predict the ith observation
    ith_pred_prob = model_LOO.predict(X_test)
    ith_pred_value = ith_pred_prob.values[0]

    # Determine the predicted class based on the probability
    if ith_pred_value > 0.5:
        ith_pred = 1
    else:
        ith_pred = 0

    # Check if the prediction was correct 1 represents error
    actual_value = y.iloc[i]
    if ith_pred == actual_value:
        error = 0
    else:
        error = 1

    # Append the error value to results
    results.append(error)

print(results[:100])

[1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1]


In [24]:
# e
# Compute LOO error
Avg_loo_error = np.mean(results)
print(Avg_loo_error)

0.44995408631772266


The results appear to be poor. The averaged test error over the dataset for LOOCV is .45 which indicates that it has approximately 50% chance of being correct for a binary classification.